In [1]:
import pandas as pd
import datetime

In [2]:
df_vp = pd.read_parquet("/home/donsson/proyectos/API/ventas_perdidas_2025.parquet")

# REGLAS CONTAR VP

In [3]:
import pandas as pd
import numpy as np

# ===============================
# Filtrar almacenamiento agotado
# ===============================
df_vp = df_vp[df_vp["almacenamiento_tipo"].str.lower() == "agotado"]

# ===============================
# Asegurar tipos correctos
# ===============================
df_vp = df_vp.copy()
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"], errors="coerce")

# Numéricos
for col in ["cantidad", "cantidad_existencia", "cantidad_reservada"]:
    df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").fillna(0).clip(lower=0)

# ===============================
# Reglas Odoo vectorizadas
# ===============================
is_cot = df_vp["origen"].fillna("").str.lower() == "cotizacion"
ignore_mask = df_vp["cantidad"] >= 100

ajuste = np.where(
    is_cot,
    df_vp["cantidad"] - df_vp["cantidad_existencia"] - df_vp["cantidad_reservada"],
    df_vp["cantidad"] - df_vp["cantidad_reservada"]
)

# Aplicar reglas de descarte y piso en cero
ajuste = np.where(ignore_mask, 0, ajuste)
ajuste = np.where(ajuste > 0, ajuste, 0)

df_vp["ventas_perdidas"] = ajuste.astype(float)

# ===============================
# Columnas temporales
# ===============================
df_vp["Semana"] = df_vp["fecha"].dt.to_period("W").dt.start_time
df_vp["ano"]   = df_vp["Semana"].dt.year
df_vp["mes"]   = df_vp["Semana"].dt.month
df_vp["dia"]   = df_vp["Semana"].dt.day

# ===============================
# Filtro adicional: excluir SERV y CARCASA
# ===============================
mask_excluir = ~df_vp["product_ref"].str.contains("SERV|CARCASA", case=False, na=False)
df_vp = df_vp[mask_excluir]

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
lost_by_week = (
    df_vp.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"])
    .agg(
        lost_sales=("ventas_perdidas", "sum"),   # suma total de ventas perdidas
        veces_vp=("ventas_perdidas", "count")    # número de veces que hubo pérdida
    )
    .reset_index()
)

# Mostrar resultado agrupado
vp_week = lost_by_week


In [4]:
vp_week.to_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/exceles/vp_año.xlsx")

In [5]:
mes = "TODOS"
vp = vp_week[(vp_week["ano"]==2025)]
vp.to_excel(f"/home/donsson/proyectos/MODELO ABASTECIMIENTO/exceles/vp_meses/vp_definitivasparaanalisis{mes}.xlsx")


In [7]:
df_vp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58271 entries, 0 to 58284
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   cantidad_reservada   58271 non-null  int64         
 1   product_ref          58271 non-null  object        
 2   cantidad             58271 non-null  int64         
 3   almacenamiento_tipo  58271 non-null  object        
 4   fecha                58271 non-null  datetime64[ns]
 5   cantidad_existencia  58271 non-null  int64         
 6   origen               58271 non-null  object        
 7   store_name           58271 non-null  object        
 8   store_id_num         58271 non-null  int64         
 9   cliente_name         58271 non-null  object        
 10  cliente_id_num       58271 non-null  int64         
 11  ventas_perdidas      58271 non-null  float64       
 12  Semana               58271 non-null  datetime64[ns]
 13  ano                  58271 non-null 

In [9]:
agrupado = df_vp.groupby(["ano", "mes", "store_name", "product_ref"]).agg({
    "ventas_perdidas": "sum",
    "cantidad": "sum",             # asumimos que son ventas en stock
    "cantidad_reservada": "sum"    # ejemplo: podría ser ventas perdidas en stock
}).reset_index()

# Calcular NV
agrupado["NV"] = 1 - (agrupado["ventas_perdidas"] /
                      (agrupado["ventas_perdidas"] + 
                       agrupado["cantidad"] + 
                       agrupado["cantidad_reservada"]))


In [10]:
agrupado

,ano,mes,store_name,product_ref,ventas_perdidas,cantidad,cantidad_reservada,NV
0,2024,11,PRINCIPAL COTA,BLS00001125,0.0,1,2,1.000
1,2024,11,SUCURSAL BARRANQUILLA,BAC00115125,1.0,1,0,0.500
2,2024,11,SUCURSAL BARRANQUILLA,BAC00200125,1.0,1,0,0.500
3,2024,11,SUCURSAL BARRANQUILLA,BAC00201125,1.0,1,0,0.500
4,2024,11,SUCURSAL BARRANQUILLA,BAC00202125,1.0,1,0,0.500
...,...,...,...,...,...,...,...,...
33825,2025,9,SUCURSAL VALLADOLID,DLS00311189,0.0,1,36,1.000
33826,2025,9,SUCURSAL VALLADOLID,DLS00532189,0.0,1,6,1.000
33827,2025,9,SUCURSAL VALLADOLID,DLS00665189,0.0,1,1,1.000
33828,2025,9,SUCURSAL VALLADOLID,DLS10286189,1.0,1,0,0.500
